# Estimate Time Request for Key Project Proposal

This notebook summarizes the results of simulating the proposed follow-up strategy for targets from the different surveys, and computes the overal total observing time requested on LCO facilities. 

For details of the calculations of total numbers of targets selected from each survey, and the corresponding observing time calculations, please see notebooks:
* ./gaia/estimated_number_gaia_events2.ipynb
* ./ztf/estimated_number_ztf_events2.ipynb
* ./ogle/estimated_number_bulge_events2.ipynb

## Summary of time for each element of observations

Here we summarize the findings of the above notebooks, in terms of the expected numbers of targets and the observing time required to complete the proposed strategy on each instrument class, as required by the proposal.  

Note that there may be slight differences in the total time estimated, as the previous notebooks allowed decimal numbers of targets whereas here we require integers. 

In [1]:
import numpy as np

In [223]:
class TargetSet:
    def __init__(self):
        self.survey = None
        self.survey_years = []
        self.survey_semesters = {2023: ['B'], 
                                 2024: ['A', 'B'],
                                 2025: ['A', 'B'],
                                 2026: ['A']}
        self.semester_fraction = []
        self.n_planet_events_per_year = 0
        self.n_stellar_events_per_year = 0
        self.n_bh_events_per_year = 0
        
        self.sinistro_time_per_planet = 0
        self.sinistro_time_per_star = 0
        self.sinistro_time_per_bh = 0
        
        self.floyds_frac = 0.0
        self.floyds_time_per_target = 3.83 # hrs
        
        self.time_per_caustic = 6.0*2.0 # hrs, with two caustic crossings per event
        self.time_per_anomaly = 4.0 # hrs
        self.vis_fraction = 0.0
    
    def calc_total_n_events(self):
        """Calculate the total number of events over the lifetime of the survey operations"""
        self.n_planet_events = 0
        self.n_stellar_events = 0
        self.n_bh_events = 0 
        
        for i,year in enumerate(self.survey_semesters.keys()):
            if len(self.survey_semesters[year]) == 2:
                self.n_planet_events += self.n_planet_events_per_year
                self.n_stellar_events += self.n_stellar_events_per_year
                self.n_bh_events += self.n_bh_events_per_year
                
            elif len(self.survey_semesters[year]) == 1:
                if self.survey_semesters[year][0] == 'A':
                    self.n_planet_events += self.n_planet_events_per_year * self.semester_fraction[1]
                    self.n_stellar_events += self.n_stellar_events_per_year * self.semester_fraction[1]
                    self.n_bh_events += self.n_bh_events_per_year * self.semester_fraction[1]
                else:
                    self.n_planet_events += self.n_planet_events_per_year * self.semester_fraction[0]
                    self.n_stellar_events += self.n_stellar_events_per_year * self.semester_fraction[0]
                    self.n_bh_events += self.n_bh_events_per_year * self.semester_fraction[0]
                
        print('Expect ~'+str(self.n_planet_events_per_year)+' planetary events per year, and '+str(self.n_planet_events)+' over the remaining survey lifetime')
        print('Expect ~'+str(self.n_stellar_events_per_year)+' stellar events per year, and '+str(self.n_stellar_events)+' over the remaining survey lifetime')
        print('Expect ~'+str(self.n_bh_events_per_year)+' stellar remnant events per year, and '+str(self.n_bh_events)+' over the remaining survey lifetime')
      
    def calc_sinistro_time(self):
        """Calculate the time required on the 1m/Sinistro network for different categories of events"""
        
        self.sinistro_time_planet_per_year = self.sinistro_time_per_planet * self.n_planet_events_per_year
        self.sinistro_time_stellar_per_year = self.sinistro_time_per_star * self.n_stellar_events_per_year
        self.sinistro_time_bh_per_year = self.sinistro_time_per_bh * self.n_bh_events_per_year
        print('Time on 1m/Sinistro network required for planetary events per year = '+str(self.sinistro_time_planet_per_year)+'hrs')
        print('Time on 1m/Sinistro network required for stellar events per year = '+str(self.sinistro_time_stellar_per_year)+'hrs')
        print('Time on 1m/Sinistro network required for stellar remnant events per year = '+str(self.sinistro_time_bh_per_year)+'hrs')
        
        self.sinistro_time_planet = []
        self.sinistro_time_stellar = []
        self.sinistro_time_bh = []
        self.sinistro_time = []
        for i,year in enumerate(self.survey_years):
            #self.sinistro_time_planet.append(self.sinistro_time_planet_per_year)
            #self.sinistro_time_stellar.append(self.sinistro_time_stellar_per_year)
            #self.sinistro_time_bh.append(self.sinistro_time_bh_per_year)
            
            time_year = self.sinistro_time_planet_per_year \
                        + self.sinistro_time_stellar_per_year \
                        + self.sinistro_time_bh_per_year
            print('Total time on 1m/Sinistro in year '+str(year)+': '+str(time_year))
            self.sinistro_time.append(time_year)
        
        (time_planet,time_stellar,time_bh,time_total) = self.calc_instrument_time_per_semester('1m/Sinistro',
                                                                                    self.sinistro_time_planet_per_year, 
                                                                                    self.sinistro_time_stellar_per_year, 
                                                                                    self.sinistro_time_bh_per_year)
        self.sinistro_time_planet = time_planet
        self.sinistro_time_stellar = time_stellar
        self.sinistro_time_bh = time_bh
        self.sinistro_time = time_total
        
    def calc_instrument_time_per_semester(self,inst_name,inst_time_planet_per_year, 
                                          inst_time_stellar_per_year, inst_time_bh_per_year,
                                          planets=True, stellar=True):
        
        inst_time_planet = []
        inst_time_stellar = []
        inst_time_bh = []
        inst_time_total = []
        for i,year in enumerate(self.survey_semesters.keys()):
            for s in self.survey_semesters[year]:
                if s == 'A':
                    if planets:
                        inst_time_planet.append(inst_time_planet_per_year*self.semester_fraction[1])
                    else:
                        inst_time_planet.append(0.0)
                    if stellar:
                        inst_time_stellar.append(inst_time_stellar_per_year*self.semester_fraction[1])
                    else:
                        inst_time_stellar.append(0.0)
                    inst_time_bh.append(inst_time_bh_per_year*self.semester_fraction[1])
                else:
                    if planets:
                        inst_time_planet.append(inst_time_planet_per_year*self.semester_fraction[0])
                    else:
                        inst_time_planet.append(0.0)
                    if stellar:
                        inst_time_stellar.append(inst_time_stellar_per_year*self.semester_fraction[0])
                    else:
                        inst_time_stellar.append(0.0)
                    inst_time_bh.append(inst_time_bh_per_year*self.semester_fraction[0])
                
                time_semester = inst_time_planet[-1]+inst_time_stellar[-1]+inst_time_bh[-1]
                inst_time_total.append(time_semester)
                print('Time on '+inst_name+' in semester '+str(year)+s+': '+str(time_semester))

 #           if len(self.survey_semesters[year]) == 2:
 #               if planets:
 #                   inst_time_planet.append(inst_time_planet_per_year)
 #                   inst_time_planet.append(inst_time_planet_per_year)
 #               else:
 ##                   inst_time_planet.append(0.0)
  #                  inst_time_planet.append(0.0)
  #              if stellar:
  #                  inst_time_stellar.append(inst_time_stellar_per_year)
  #                  inst_time_stellar.append(inst_time_stellar_per_year)
  #              else:
  #                  inst_time_stellar.append(0.0)
  #                  inst_time_stellar.append(0.0)
  #              inst_time_bh.append(inst_time_bh_per_year)
  #              inst_time_bh.append(inst_time_bh_per_year)
                
  #              time_semester = inst_time_planet[-1]+inst_time_stellar[-1]+inst_time_bh[-1]
  #              inst_time_total.append(time_semester)
  #              print('Time on '+inst_name+' in semesters '+str(year)+'A+B: '+str(time_semester))
            
   #         elif len(self.survey_semesters[year]) == 1:
   #             if self.survey_semesters[year][0] == 'A':
   #                 if planets:
   #                     inst_time_planet.append(inst_time_planet_per_year*self.semester_fraction[1])
   #                 else:
   #                     inst_time_planet.append(0.0)
   #                 if stellar:
   #                     inst_time_stellar.append(inst_time_stellar_per_year*self.semester_fraction[1])
   #                 else:
   #                     inst_time_stellar.append(0.0)
    #                inst_time_bh.append(inst_time_bh_per_year*self.semester_fraction[1])
    #            else:
    #                if planets:
   #                     inst_time_planet.append(inst_time_planet_per_year*self.semester_fraction[0])
   #                 else:
   #                     inst_time_planet.append(0.0)
   #                 if stellar:
   #                     inst_time_stellar.append(inst_time_stellar_per_year*self.semester_fraction[0])
   #                 else:
   #                     inst_time_stellar.append(0.0)
   #                 inst_time_bh.append(inst_time_bh_per_year*self.semester_fraction[0])
                
   #             time_semester = inst_time_planet[-1]+inst_time_stellar[-1]+inst_time_bh[-1]
   #             inst_time_total.append(time_semester)
   #             print('Time on '+inst_name+' in semester '+str(year)+self.survey_semesters[year][0]+': '+str(time_semester))

        return inst_time_planet, inst_time_stellar, inst_time_bh, inst_time_total
                
    def calc_floyds_time(self, planets=True, stellar=True):
        """Calculate the time required for 2m/FLOYDS spectroscopy"""
        
        self.floyds_bh_events_per_year = int(round(self.floyds_frac*self.n_bh_events_per_year,0))
        print('Number of stellar remnant events bright enough for FLOYDS per year = '+str(self.floyds_bh_events_per_year))
        self.floyds_time_bh_per_year = int(round((self.floyds_bh_events_per_year * self.floyds_time_per_target),0))
        print('Time on 2m/FLOYDS network required for stellar remnant events per year = '+str(self.floyds_time_bh_per_year)+'hrs')

        if planets:
            self.floyds_planet_events_per_year = int(round(self.floyds_frac*self.n_planet_events_per_year,0))
            print('Number of planetary events bright enough for FLOYDS per year = '+str(self.floyds_planet_events_per_year))
            
            self.floyds_time_planet_per_year = int(round((self.floyds_planet_events_per_year * self.floyds_time_per_target),0))
            print('Time on 2m/FLOYDS network required for planetary events per year = '+str(self.floyds_time_planet_per_year)+'hrs')
        else:
            self.floyds_time_planet_per_year = 0
            
        if stellar:
            self.floyds_stellar_events_per_year = int(round(self.floyds_frac*self.n_stellar_events_per_year,0))
            print('Number of stellar events bright enough for FLOYDS per year = '+str(self.floyds_stellar_events_per_year))
        
            self.floyds_time_stellar_per_year = int(round((self.floyds_stellar_events_per_year * self.floyds_time_per_target),0))
            print('Time on 2m/FLOYDS network required for stellar events per year = '+str(self.floyds_time_stellar_per_year)+'hrs')
        else:
            self.floyds_time_stellar_per_year = 0
            
        self.floyds_time_planet = []
        self.floyds_time_stellar = []
        self.floyds_time_bh = []
        self.floyds_time = []
        for i,year in enumerate(self.survey_years):
#            if planets:
#                self.floyds_time_planet.append(self.floyds_time_planet_per_year)
#            else:
#                self.floyds_time_planet.append(0.0)
            
#            if stellar:
#                self.floyds_time_stellar.append(self.floyds_time_stellar_per_year)
#            else:
#                self.floyds_time_stellar.append(0.0)
                                                
#            self.floyds_time_bh.append(self.floyds_time_bh_per_year)
            
            time_year = self.floyds_time_bh_per_year
            if planets:
                time_year += self.floyds_time_planet_per_year
            if stellar:
                time_year += self.floyds_time_stellar_per_year
                
            self.floyds_time.append(time_year)
            print('Total time on 2m/FLOYDS in year '+str(year)+': '+str(time_year))

        (time_planet,time_stellar,time_bh,time_total) = self.calc_instrument_time_per_semester('2m/FLOYDS',
                                                                                    self.floyds_time_planet_per_year, 
                                                                                    self.floyds_time_stellar_per_year, 
                                                                                    self.floyds_time_bh_per_year,
                                                                                    planets=planets,
                                                                                    stellar=stellar)
        self.floyds_time_planet = time_planet
        self.floyds_time_stellar = time_stellar
        self.floyds_time_bh = time_bh
        self.floyds_time = time_total

    def calc_2m_imagers_time(self, planets=True, stellar=True):
        """Estimate of time required in hours on the 2m/MusCAT or 2m/Spectral network to 
        observe caustic crossings for stellar & planetary binary events in RAPID RESPONSE MODE"""

        self.time_rr_stellar_events = []
        self.time_rr_planet_events = []
        self.time_2m_imagers = []
        for i,year in enumerate(self.survey_years):
            time_year = 0.0
            if stellar:
                n_stellar_per_year = int(round((self.n_stellar_events_per_year * self.vis_fraction),0))
                time_stellar_year = n_stellar_per_year * self.time_per_caustic
                self.time_rr_stellar_events.append(time_stellar_year)
                time_year += time_stellar_year
                print('Time on 2m/imagers for '+str(n_stellar_per_year)+' stellar events in year '+str(year)+': '+str(time_stellar_year))
            else:
                self.time_rr_stellar_events.append(0.0)
            
            if planets:
                n_planet_per_year = int(round((self.n_planet_events_per_year * self.vis_fraction),0))
                time_planet_year = n_planet_per_year * self.time_per_caustic
                self.time_rr_planet_events.append(time_planet_year)
                time_year += time_planet_year
                print('Time on 2m/imagers for '+str(n_planet_per_year)+' planetary events in year '+str(year)+': '+str(time_planet_year))
            else:
                self.time_rr_planet_events.append(0.0)
                
            self.time_2m_imagers.append(time_year)
            print('Total time on 2m/imagers in RR mode in year '+str(year)+': '+str(time_year))
    
        n_planet_per_year = int(round((self.n_planet_events_per_year * self.vis_fraction),0))
        time_planet_year = n_planet_per_year * self.time_per_caustic
        n_stellar_per_year = int(round((self.n_stellar_events_per_year * self.vis_fraction),0))
        time_stellar_year = n_stellar_per_year * self.time_per_caustic
        (time_planet,time_stellar,time_bh,time_total) = self.calc_instrument_time_per_semester('2m/imagers',
                                                                                    time_planet_year, 
                                                                                    time_stellar_year, 
                                                                                    0,
                                                                                    planets=planets,
                                                                                    stellar=stellar)
        self.time_rr_planet_events = time_planet
        self.time_rr_stellar_events = time_stellar
        self.time_2m_imagers = time_total
    
    def calc_1m_rr_time(self, planets=True, stellar=True):
        """Time required in Rapid Reponse mode from the 1m network is equivalent to the fraction of the time 
        dedicated to observing anomalies and caustic crossings that isn't already covered by the 2m. 
        
        It is only calculated for the planetary and stellar binary events to be followed from each survey. 
        
        """
        
        # The fraction of the Rapid Response time from the 1m network
        rr_fraction = 1.0 - self.vis_fraction
        
        self.time_rr_stellar_events_1m = []
        self.time_rr_planet_events_1m = []
        self.time_rr_1m_imagers = []
        for i,year in enumerate(self.survey_years):
            time_year = 0.0
            if stellar:
                n_stellar_per_year = int(round((self.n_stellar_events_per_year * rr_fraction),0))
                time_stellar_year = n_stellar_per_year * self.time_per_caustic
                self.time_rr_stellar_events_1m.append(time_stellar_year)
                time_year += time_stellar_year
                print('Rapid response time on 1m/imagers for '+str(n_stellar_per_year)+' stellar events in year '+str(year)+': '+str(time_stellar_year))
            else:
                self.time_rr_stellar_events_1m.append(0.0)
            
            if planets:
                n_planet_per_year = int(round((self.n_planet_events_per_year * rr_fraction),0))
                time_planet_year = n_planet_per_year * self.time_per_caustic
                self.time_rr_planet_events_1m.append(time_planet_year)
                time_year += time_planet_year
                print('Rapid response time on 1m/imagers for '+str(n_planet_per_year)+' planetary events in year '+str(year)+': '+str(time_planet_year))
            else:
                self.time_rr_planet_events_1m.append(0.0)
                
            self.time_rr_1m_imagers.append(time_year)
            print('Total time on 1m/imagers in RR mode in year '+str(year)+': '+str(time_year))
        
        
        if stellar:
            n_stellar_per_year = int(round((self.n_stellar_events_per_year * rr_fraction),0))
            time_stellar_year = n_stellar_per_year * self.time_per_caustic
        else:
            time_stellar_year = 0.0
        
        if planets:
            n_planet_per_year = int(round((self.n_planet_events_per_year * rr_fraction),0))
            time_planet_year = n_planet_per_year * self.time_per_caustic
        else:
            time_planet_year = 0.0
            
        (time_planet,time_stellar,time_bh,time_total) = self.calc_instrument_time_per_semester('1m/imagers RR',
                                                                                    time_planet_year, 
                                                                                    time_stellar_year, 
                                                                                    0,
                                                                                    planets=planets,
                                                                                    stellar=stellar)
        self.time_rr_stellar_events_1m = time_planet
        self.time_rr_planet_events_1m = time_stellar
        self.time_rr_1m_imagers = time_total
        
    def calc_time_totals(self):
        # Calculate time required over all survey semesters
        self.sinistro_time_total = np.array(self.sinistro_time).sum()
        self.floyds_time_total = np.array(self.floyds_time).sum()
        self.time_2m_imagers_total = np.array(self.time_2m_imagers).sum()
        
        self.sinistro_1m_queue = np.zeros(len(self.sinistro_time))
        s = -1
        for i,year in enumerate(self.survey_semesters.keys()):
            annual_semesters = self.survey_semesters[year]
            
            for sem in annual_semesters:
                s += 1
                self.sinistro_1m_queue[s] = self.sinistro_time[s] - self.time_rr_1m_imagers[s]
                print('Semester '+str(year)+sem+' Sinistro total '+str(self.sinistro_time[i]))
                print(' -> Sinistro queue mode '+str(self.sinistro_1m_queue[s]))
                print(' -> Sinistro RR mode '+str(self.time_rr_1m_imagers[s]))
        
        # Calculate time required per semester in each survey year
        # LCO Semesters run A: Feb 1 - July 31 and B: Aug 1 - Jan 31
        # semester_fraction refers to the ratio of the annual target count that 
        # expected to be found in the [B, A] semesters. 
        # The Key Project proposals run from 2023B - 2026A inclusive
        
        print('\n')
        print('Sinistro time total over project: '+str(self.sinistro_time_total)+'hrs')
        print('FLOYDS time total over project: '+str(self.floyds_time_total)+'hrs')
        print('2m imagers time total over project: '+str(self.time_2m_imagers_total)+'hrs')
        
    def calc_time_per_semester(self, annual_time):
        
        key_project_semesters = {}
        
        for i,year in enumerate(self.survey_semesters.keys()):
            annual_semesters = self.survey_semesters[year]
            
            Atime = annual_time[i] * self.semester_fraction[1]
            Btime = annual_time[i] * self.semester_fraction[0]
            
            semester_time = {}
            for s in annual_semesters:
                if s == 'A':
                    semester_time[s] = Atime
                else:
                    semester_time[s] = Btime
                    
            key_project_semesters[year] = semester_time
            print('Year '+str(year)+': '+repr(key_project_semesters[year]))
        
        return key_project_semesters
        

### Gaia Targets

The simulation considers LCO observations of stellar remnant and stellar (most likely single-lens) events detected by Gaia over its last two years of operation (2023-2025). 

Note that this does not preclude observing a stellar or planetary binary of interest if one is discovered, but for the purpose of the proposal I have aimed to be conservative. 

In [224]:
gaia_targets = TargetSet()

In [225]:
# Note the 'planets' category here is used to refer to any binary event, since Gaia binaries are likely to be outside
# the Bulge and therefore high priority, while the stellar category here refers to single-lens stellar mass lenses. 
# The 'planet' events per year are subtracted from the overall total number of events expected per year to avoid 
# double counting
# The semester fraction refers to the division of targets per year into the A and B semesters, which for Gaia is 50:50
gaia_targets.survey_years = [2023, 2024]
gaia_targets.survey_semesters = {2023: ['B'], 
                                 2024: ['A', 'B'],
                                 2025: ['A']}
gaia_targets.semester_fraction = [0.5, 0.5]
gaia_targets.n_planet_events_per_year = 9
gaia_targets.n_stellar_events_per_year = 92 - gaia_targets.n_planet_events_per_year
gaia_targets.n_bh_events_per_year = 7
gaia_targets.calc_total_n_events()      


Expect ~9 planetary events per year, and 18.0 over the remaining survey lifetime
Expect ~83 stellar events per year, and 166.0 over the remaining survey lifetime
Expect ~7 stellar remnant events per year, and 14.0 over the remaining survey lifetime


In [226]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
gaia_targets.sinistro_time_per_planet = 8+12
gaia_targets.sinistro_time_per_star = 4.5
gaia_targets.sinistro_time_per_bh = 6.2
gaia_targets.calc_sinistro_time()


Time on 1m/Sinistro network required for planetary events per year = 180hrs
Time on 1m/Sinistro network required for stellar events per year = 373.5hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 43.4hrs
Total time on 1m/Sinistro in year 2023: 596.9
Total time on 1m/Sinistro in year 2024: 596.9
Time on 1m/Sinistro in semester 2023B: 298.45
Time on 1m/Sinistro in semester 2024A: 298.45
Time on 1m/Sinistro in semester 2024B: 298.45
Time on 1m/Sinistro in semester 2025A: 298.45


In [227]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
gaia_targets.floyds_frac = 0.994 # Fraction of events likely to be bright enough for FLOYDS
gaia_targets.calc_floyds_time(stellar=False)


Number of stellar remnant events bright enough for FLOYDS per year = 7
Time on 2m/FLOYDS network required for stellar remnant events per year = 27hrs
Number of planetary events bright enough for FLOYDS per year = 9
Time on 2m/FLOYDS network required for planetary events per year = 34hrs
Total time on 2m/FLOYDS in year 2023: 61
Total time on 2m/FLOYDS in year 2024: 61
Time on 2m/FLOYDS in semester 2023B: 30.5
Time on 2m/FLOYDS in semester 2024A: 30.5
Time on 2m/FLOYDS in semester 2024B: 30.5
Time on 2m/FLOYDS in semester 2025A: 30.5


In [228]:
# Estimate of time required in hours on the 2m/MusCAT or 2m/Spectral network to 
# observe caustic crossings for stellar binary events in RAPID RESPONSE MODE
# Note 'stellar' for this function refers to all single stellar events
gaia_targets.vis_fraction = 8.0 / 24.0
gaia_targets.calc_2m_imagers_time(stellar=False)



Time on 2m/imagers for 3 planetary events in year 2023: 36.0
Total time on 2m/imagers in RR mode in year 2023: 36.0
Time on 2m/imagers for 3 planetary events in year 2024: 36.0
Total time on 2m/imagers in RR mode in year 2024: 36.0
Time on 2m/imagers in semester 2023B: 18.0
Time on 2m/imagers in semester 2024A: 18.0
Time on 2m/imagers in semester 2024B: 18.0
Time on 2m/imagers in semester 2025A: 18.0


In [229]:
# Estimate the remaining Rapid Response time required from the 1m network
gaia_targets.calc_1m_rr_time(stellar=False)

Rapid response time on 1m/imagers for 6 planetary events in year 2023: 72.0
Total time on 1m/imagers in RR mode in year 2023: 72.0
Rapid response time on 1m/imagers for 6 planetary events in year 2024: 72.0
Total time on 1m/imagers in RR mode in year 2024: 72.0
Time on 1m/imagers RR in semester 2023B: 36.0
Time on 1m/imagers RR in semester 2024A: 36.0
Time on 1m/imagers RR in semester 2024B: 36.0
Time on 1m/imagers RR in semester 2025A: 36.0


In [230]:
gaia_targets.calc_time_totals()

Semester 2023B Sinistro total 298.45
 -> Sinistro queue mode 262.45
 -> Sinistro RR mode 36.0
Semester 2024A Sinistro total 298.45
 -> Sinistro queue mode 262.45
 -> Sinistro RR mode 36.0
Semester 2024B Sinistro total 298.45
 -> Sinistro queue mode 262.45
 -> Sinistro RR mode 36.0
Semester 2025A Sinistro total 298.45
 -> Sinistro queue mode 262.45
 -> Sinistro RR mode 36.0


Sinistro time total over project: 1193.8hrs
FLOYDS time total over project: 122.0hrs
2m imagers time total over project: 72.0hrs


### ZTF Targets

Similarly to Gaia events, the simulation assumes that ZTF is most likely to detect stellar remnant and single-lens stellar events.  

In [231]:
ztf_targets = TargetSet()

In [233]:
ztf_targets.survey_years = [2023, 2024, 2025]
ztf_targets.survey_semesters = {2023: ['B'], 
                                 2024: ['A', 'B'],
                                 2025: ['A', 'B'],
                                 2026: ['A']}
ztf_targets.semester_fraction = [0.5, 0.5]
ztf_targets.n_planet_events_per_year = 0.34
ztf_targets.n_stellar_events_per_year = 14
ztf_targets.n_bh_events_per_year = 1
ztf_targets.calc_total_n_events()

Expect ~0.34 planetary events per year, and 1.02 over the remaining survey lifetime
Expect ~14 stellar events per year, and 42.0 over the remaining survey lifetime
Expect ~1 stellar remnant events per year, and 3.0 over the remaining survey lifetime


In [234]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
ztf_targets.sinistro_time_per_planet = 6+12
ztf_targets.sinistro_time_per_star = 4.2
ztf_targets.sinistro_time_per_bh = 9.2
ztf_targets.calc_sinistro_time()


Time on 1m/Sinistro network required for planetary events per year = 6.12hrs
Time on 1m/Sinistro network required for stellar events per year = 58.800000000000004hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 9.2hrs
Total time on 1m/Sinistro in year 2023: 74.12
Total time on 1m/Sinistro in year 2024: 74.12
Total time on 1m/Sinistro in year 2025: 74.12
Time on 1m/Sinistro in semester 2023B: 37.06
Time on 1m/Sinistro in semester 2024A: 37.06
Time on 1m/Sinistro in semester 2024B: 37.06
Time on 1m/Sinistro in semester 2025A: 37.06
Time on 1m/Sinistro in semester 2025B: 37.06
Time on 1m/Sinistro in semester 2026A: 37.06


In [235]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
ztf_targets.floyds_frac = 0.995 # Fraction of events likely to be bright enough for FLOYDS
ztf_targets.calc_floyds_time()

Number of stellar remnant events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for stellar remnant events per year = 4hrs
Number of planetary events bright enough for FLOYDS per year = 0
Time on 2m/FLOYDS network required for planetary events per year = 0hrs
Number of stellar events bright enough for FLOYDS per year = 14
Time on 2m/FLOYDS network required for stellar events per year = 54hrs
Total time on 2m/FLOYDS in year 2023: 58
Total time on 2m/FLOYDS in year 2024: 58
Total time on 2m/FLOYDS in year 2025: 58
Time on 2m/FLOYDS in semester 2023B: 29.0
Time on 2m/FLOYDS in semester 2024A: 29.0
Time on 2m/FLOYDS in semester 2024B: 29.0
Time on 2m/FLOYDS in semester 2025A: 29.0
Time on 2m/FLOYDS in semester 2025B: 29.0
Time on 2m/FLOYDS in semester 2026A: 29.0


In [236]:
# Estimate of time required in hours on the FTN 2m/MusCAT to 
# observe caustic crossings for stellar binary events in RAPID RESPONSE MODE
ztf_targets.vis_fraction = 8.0 / 24.0
ztf_targets.calc_2m_imagers_time(stellar=False)

Time on 2m/imagers for 0 planetary events in year 2023: 0.0
Total time on 2m/imagers in RR mode in year 2023: 0.0
Time on 2m/imagers for 0 planetary events in year 2024: 0.0
Total time on 2m/imagers in RR mode in year 2024: 0.0
Time on 2m/imagers for 0 planetary events in year 2025: 0.0
Total time on 2m/imagers in RR mode in year 2025: 0.0
Time on 2m/imagers in semester 2023B: 0.0
Time on 2m/imagers in semester 2024A: 0.0
Time on 2m/imagers in semester 2024B: 0.0
Time on 2m/imagers in semester 2025A: 0.0
Time on 2m/imagers in semester 2025B: 0.0
Time on 2m/imagers in semester 2026A: 0.0


In [237]:
# Estimate the remaining Rapid Response time required from the 1m network
ztf_targets.calc_1m_rr_time(stellar=False)

Rapid response time on 1m/imagers for 0 planetary events in year 2023: 0.0
Total time on 1m/imagers in RR mode in year 2023: 0.0
Rapid response time on 1m/imagers for 0 planetary events in year 2024: 0.0
Total time on 1m/imagers in RR mode in year 2024: 0.0
Rapid response time on 1m/imagers for 0 planetary events in year 2025: 0.0
Total time on 1m/imagers in RR mode in year 2025: 0.0
Time on 1m/imagers RR in semester 2023B: 0.0
Time on 1m/imagers RR in semester 2024A: 0.0
Time on 1m/imagers RR in semester 2024B: 0.0
Time on 1m/imagers RR in semester 2025A: 0.0
Time on 1m/imagers RR in semester 2025B: 0.0
Time on 1m/imagers RR in semester 2026A: 0.0


In [238]:
ztf_targets.calc_time_totals()

Semester 2023B Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0
Semester 2024A Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0
Semester 2024B Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0
Semester 2025A Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0
Semester 2025B Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0
Semester 2026A Sinistro total 37.06
 -> Sinistro queue mode 37.06
 -> Sinistro RR mode 0.0


Sinistro time total over project: 222.36hrs
FLOYDS time total over project: 174.0hrs
2m imagers time total over project: 0.0hrs


### Bulge Targets from the extended survey region only

The simulation considers only events from a rough annulus around the central Bulge, within the OGLE extended survey region but not including those fields covered by KMTNet at high cadence.  It is hard to justify time for follow-up observations within that high cadence zone. 

The Bulge simulation considers planetary and stellar binary events only and long-timescale (>100d) single-lens events. 

In [240]:
bulge_targets = TargetSet()

In [241]:
bulge_targets.survey_years = [2023, 2024, 2025]
bulge_targets.survey_semesters = {2023: ['B'], 
                                 2024: ['A', 'B'],
                                 2025: ['A', 'B'],
                                 2026: ['A']}
# Bulge is observed 1 Feb to 30 Sept each year, so 6 out of 8 months are in the A semester
bulge_targets.semester_fraction = [(2.0/8.0), (6.0/8.0)]
bulge_targets.n_stellar_events_per_year = 0 # No single-lens stellar events will be followed
bulge_targets.n_planet_events_per_year = 9 # Includes all binary events, planetary, BD, stellar companions
bulge_targets.n_bh_events_per_year = 6
bulge_targets.calc_total_n_events()


Expect ~9 planetary events per year, and 27.0 over the remaining survey lifetime
Expect ~0 stellar events per year, and 0.0 over the remaining survey lifetime
Expect ~6 stellar remnant events per year, and 18.0 over the remaining survey lifetime


In [242]:
# Estimates of time required in hours on the 1m/Sinistro network to observe the different categories of events
bulge_targets.sinistro_time_per_planet = 21.1
#bulge_targets.sinistro_time_per_star = 18
bulge_targets.sinistro_time_per_bh = 26.4
bulge_targets.calc_sinistro_time()

Time on 1m/Sinistro network required for planetary events per year = 189.9hrs
Time on 1m/Sinistro network required for stellar events per year = 0hrs
Time on 1m/Sinistro network required for stellar remnant events per year = 158.39999999999998hrs
Total time on 1m/Sinistro in year 2023: 348.29999999999995
Total time on 1m/Sinistro in year 2024: 348.29999999999995
Total time on 1m/Sinistro in year 2025: 348.29999999999995
Time on 1m/Sinistro in semester 2023B: 87.07499999999999
Time on 1m/Sinistro in semester 2024A: 261.225
Time on 1m/Sinistro in semester 2024B: 87.07499999999999
Time on 1m/Sinistro in semester 2025A: 261.225
Time on 1m/Sinistro in semester 2025B: 87.07499999999999
Time on 1m/Sinistro in semester 2026A: 261.225


In [243]:
# Estimates of time required in hours on the 2m/FLOYDS network to observe the different categories of events
bulge_targets.floyds_frac = 0.121 # Fraction of events likely to be bright enough for FLOYDS
bulge_targets.calc_floyds_time()

Number of stellar remnant events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for stellar remnant events per year = 4hrs
Number of planetary events bright enough for FLOYDS per year = 1
Time on 2m/FLOYDS network required for planetary events per year = 4hrs
Number of stellar events bright enough for FLOYDS per year = 0
Time on 2m/FLOYDS network required for stellar events per year = 0hrs
Total time on 2m/FLOYDS in year 2023: 8
Total time on 2m/FLOYDS in year 2024: 8
Total time on 2m/FLOYDS in year 2025: 8
Time on 2m/FLOYDS in semester 2023B: 2.0
Time on 2m/FLOYDS in semester 2024A: 6.0
Time on 2m/FLOYDS in semester 2024B: 2.0
Time on 2m/FLOYDS in semester 2025A: 6.0
Time on 2m/FLOYDS in semester 2025B: 2.0
Time on 2m/FLOYDS in semester 2026A: 6.0


In [244]:
# Estimate of time required in hours on the FTS 2m/Spectral or MusCAT to 
# observe caustic crossings for planetary and stellar binary events in RAPID RESPONSE MODE
bulge_targets.vis_fraction = 0.42
bulge_targets.calc_2m_imagers_time()


Time on 2m/imagers for 0 stellar events in year 2023: 0.0
Time on 2m/imagers for 4 planetary events in year 2023: 48.0
Total time on 2m/imagers in RR mode in year 2023: 48.0
Time on 2m/imagers for 0 stellar events in year 2024: 0.0
Time on 2m/imagers for 4 planetary events in year 2024: 48.0
Total time on 2m/imagers in RR mode in year 2024: 48.0
Time on 2m/imagers for 0 stellar events in year 2025: 0.0
Time on 2m/imagers for 4 planetary events in year 2025: 48.0
Total time on 2m/imagers in RR mode in year 2025: 48.0
Time on 2m/imagers in semester 2023B: 12.0
Time on 2m/imagers in semester 2024A: 36.0
Time on 2m/imagers in semester 2024B: 12.0
Time on 2m/imagers in semester 2025A: 36.0
Time on 2m/imagers in semester 2025B: 12.0
Time on 2m/imagers in semester 2026A: 36.0


In [245]:
# Estimate the remaining Rapid Response time required from the 1m network
bulge_targets.calc_1m_rr_time()

Rapid response time on 1m/imagers for 0 stellar events in year 2023: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2023: 60.0
Total time on 1m/imagers in RR mode in year 2023: 60.0
Rapid response time on 1m/imagers for 0 stellar events in year 2024: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2024: 60.0
Total time on 1m/imagers in RR mode in year 2024: 60.0
Rapid response time on 1m/imagers for 0 stellar events in year 2025: 0.0
Rapid response time on 1m/imagers for 5 planetary events in year 2025: 60.0
Total time on 1m/imagers in RR mode in year 2025: 60.0
Time on 1m/imagers RR in semester 2023B: 15.0
Time on 1m/imagers RR in semester 2024A: 45.0
Time on 1m/imagers RR in semester 2024B: 15.0
Time on 1m/imagers RR in semester 2025A: 45.0
Time on 1m/imagers RR in semester 2025B: 15.0
Time on 1m/imagers RR in semester 2026A: 45.0


In [246]:
bulge_targets.calc_time_totals()

Semester 2023B Sinistro total 87.07499999999999
 -> Sinistro queue mode 72.07499999999999
 -> Sinistro RR mode 15.0
Semester 2024A Sinistro total 261.225
 -> Sinistro queue mode 216.22500000000002
 -> Sinistro RR mode 45.0
Semester 2024B Sinistro total 261.225
 -> Sinistro queue mode 72.07499999999999
 -> Sinistro RR mode 15.0
Semester 2025A Sinistro total 87.07499999999999
 -> Sinistro queue mode 216.22500000000002
 -> Sinistro RR mode 45.0
Semester 2025B Sinistro total 87.07499999999999
 -> Sinistro queue mode 72.07499999999999
 -> Sinistro RR mode 15.0
Semester 2026A Sinistro total 261.225
 -> Sinistro queue mode 216.22500000000002
 -> Sinistro RR mode 45.0


Sinistro time total over project: 1044.9hrs
FLOYDS time total over project: 24.0hrs
2m imagers time total over project: 144.0hrs


## Maximal targets observed

If all planned observations execute completely, we should characterize the following target set over the whole project lifetime.  

Note that stellar remnant events are sufficiently long in timescale that Gaia and ZTF may both detect the same candidates.  For that reason, I've avoided double-counting them, though potentially this may be an under-estimate.  


In [247]:
total_planets = gaia_targets.n_planet_events + ztf_targets.n_planet_events + bulge_targets.n_planet_events
total_stars = gaia_targets.n_stellar_events + ztf_targets.n_stellar_events + bulge_targets.n_stellar_events
total_bh = max(gaia_targets.n_bh_events, ztf_targets.n_bh_events)
total_bh = max(total_bh, bulge_targets.n_bh_events)

print(str(total_planets)+' planetary events')
print(str(total_stars)+' stellar binary events')
print(str(total_bh)+' stellar remnant event candidates')


46.019999999999996 planetary events
208.0 stellar binary events
18.0 stellar remnant event candidates


## Total Observing Time Required

Various alternative strategies are considered here, as the total is determined by the full range of the targets we expect to study.   

### Maximal Strategy

Queue-mode photometry and spectroscopy of all observable stellar and stellar remnant lenses from Gaia, ZTF and the Bulge as well as planetary events from the extended survey region in the Bulge (outside the KMTNet high cadence zone).

Rapid response observations on the 2m imagers including MusCAT for planetary anomalies and stellar caustic crossings.

In [254]:
print('Time per instrument class per year:')
key_project_semesters = {2023: ['B'], 
                         2024: ['A', 'B'],
                         2025: ['A', 'B'],
                         2026: ['A']}

sinistro_time = np.zeros(6)
sinistro_queue = np.zeros(6)
sinistro_rr = np.zeros(6)
floyds_time = np.zeros(6)
ft_imagers_time = np.zeros(6)

s = -1
for i,year in enumerate(key_project_semesters.keys()):
    for semester in key_project_semesters[year]:
        s += 1
        for targets in [gaia_targets, ztf_targets, bulge_targets]:
            #print(targets.survey_semesters)
            if year in targets.survey_semesters.keys():
                if semester in targets.survey_semesters[year]:
                    sinistro_time[s] += targets.sinistro_time[s]
                    sinistro_queue[s] += targets.sinistro_1m_queue[s]
                    sinistro_rr[s] += targets.time_rr_1m_imagers[s]
                    floyds_time[s] += targets.floyds_time[s]
                    ft_imagers_time[s] += targets.time_2m_imagers[s]
            
        print('Semester '+str(year)+semester+' 1m/Sinistro time = '+str(int(round(sinistro_time[s],0)))+'hrs')
        print('-> Queue time = '+str(int(round(sinistro_queue[s],0)))+'hrs')
        print('-> Rapid response time = '+str(int(round(sinistro_rr[s],0)))+'hrs')
        print('-> 2m/FLOYDS time = '+str(int(round(floyds_time[s],0)))+'hrs')
        print('-> 2m/imagers time (RR mode) = '+str(int(round(ft_imagers_time[s],0)))+'hrs')
        print('\n')
    
print('Time per instrument over project lifetime:')
sinistro_total = int(round(np.array(sinistro_time).sum(),0))
sinistro_queue_total = int(round(sinistro_queue.sum(),0))
sinistro_rr_total = int(round(sinistro_rr.sum(),0))
floyds_total = int(round(np.array(floyds_time).sum(),0))
ft_imagers_total = int(round(np.array(ft_imagers_time).sum(),0))
project_total = sinistro_total + floyds_total + ft_imagers_total

print('1m/Sinistro: '+str(sinistro_total)+'hrs')
print('-> Queue mode: '+str(sinistro_queue_total)+'hrs')
print('-> Rapid response mode: '+str(sinistro_rr_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_total)+'hrs')
print('2m/imagers: '+str(ft_imagers_total)+'hrs (Rapid Response mode)\n')

print('Overall project request = '+str(project_total)+'hrs')

Time per instrument class per year:
Semester 2023B 1m/Sinistro time = 423hrs
-> Queue time = 372hrs
-> Rapid response time = 51hrs
-> 2m/FLOYDS time = 62hrs
-> 2m/imagers time (RR mode) = 30hrs


Semester 2024A 1m/Sinistro time = 597hrs
-> Queue time = 516hrs
-> Rapid response time = 81hrs
-> 2m/FLOYDS time = 66hrs
-> 2m/imagers time (RR mode) = 54hrs


Semester 2024B 1m/Sinistro time = 423hrs
-> Queue time = 372hrs
-> Rapid response time = 51hrs
-> 2m/FLOYDS time = 62hrs
-> 2m/imagers time (RR mode) = 30hrs


Semester 2025A 1m/Sinistro time = 597hrs
-> Queue time = 516hrs
-> Rapid response time = 81hrs
-> 2m/FLOYDS time = 66hrs
-> 2m/imagers time (RR mode) = 54hrs


Semester 2025B 1m/Sinistro time = 124hrs
-> Queue time = 109hrs
-> Rapid response time = 15hrs
-> 2m/FLOYDS time = 31hrs
-> 2m/imagers time (RR mode) = 12hrs


Semester 2026A 1m/Sinistro time = 298hrs
-> Queue time = 253hrs
-> Rapid response time = 45hrs
-> 2m/FLOYDS time = 35hrs
-> 2m/imagers time (RR mode) = 36hrs


Time

### Stellar remnant candidate observations only

Queue-mode photometry of stellar remnant candidates from Gaia, ZTF and the Bulge using the 1m network, plus FLOYDS spectroscopy of those that reach bright enough peak magnitudes. 

No rapid response observations.  No 2m imaging observations. 

In [ ]:
sinistro_time_per_year = gaia_targets.sinistro_time_bh_per_year \
                            + ztf_targets.sinistro_time_bh_per_year \
                            + bulge_targets.sinistro_time_bh_per_year
sinistro_time_total = gaia_targets.sinistro_time_bh \
                            + ztf_targets.sinistro_time_bh \
                            + bulge_targets.sinistro_time_bh

floyds_time_per_year = gaia_targets.floyds_time_bh_per_year \
                            + ztf_targets.floyds_time_bh_per_year \
                            + bulge_targets.floyds_time_bh_per_year
floyds_time_total = gaia_targets.floyds_time_bh \
                            + ztf_targets.floyds_time_bh \
                            + bulge_targets.floyds_time_bh

project_total = sinistro_time_total + floyds_time_total
print('Time per instrument class per year:')
print('1m/Sinistro: '+str(sinistro_time_per_year)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_per_year)+'hrs')

print('Time per instrument over project lifetime:')
print('1m/Sinistro: '+str(sinistro_time_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_total)+'hrs')

print('Overall project request = '+str(project_total)+'hrs')

## Stellar remnant and planet candidate observations only

Photometry of stellar remnant candidates from Gaia, ZTF and the Bulge using the 1m network. 

Photometry of planetary events in the Bulge extended survey region. 

FLOYDS spectroscopy of those that reach bright enough peak magnitudes.

Rapid response imaging from 2m network for planetary anomalies. 

In [ ]:
sinistro_time_per_year = gaia_targets.sinistro_time_planet_per_year + gaia_targets.sinistro_time_bh_per_year\
                            + ztf_targets.sinistro_time_planet_per_year + ztf_targets.sinistro_time_bh_per_year \
                            + bulge_targets.sinistro_time_planet_per_year + bulge_targets.sinistro_time_bh_per_year
sinistro_time_total = gaia_targets.sinistro_time_planet +  gaia_targets.sinistro_time_bh\
                            + ztf_targets.sinistro_time_planet + ztf_targets.sinistro_time_bh\
                            + bulge_targets.sinistro_time_planet + bulge_targets.sinistro_time_bh

floyds_time_per_year = gaia_targets.floyds_time_bh_per_year + gaia_targets.floyds_time_planet_per_year\
                            + ztf_targets.floyds_time_bh_per_year + ztf_targets.floyds_time_planet_per_year\
                            + bulge_targets.floyds_time_bh_per_year + bulge_targets.floyds_time_planet_per_year
floyds_time_total = gaia_targets.floyds_time_bh + gaia_targets.floyds_time_planet \
                            + ztf_targets.floyds_time_bh + ztf_targets.floyds_time_planet \
                            + bulge_targets.floyds_time_bh + bulge_targets.floyds_time_planet

# 2m imagers are only used for planetary anomalies
FT_imagers_time_per_year = gaia_targets.time_2m_imagers_planet_per_year \
                            + ztf_targets.time_2m_imagers_planet_per_year  \
                            + bulge_targets.time_2m_imagers_planet_per_year 
FT_imagers_time_total = gaia_targets.time_2m_imagers_planet  \
                            + ztf_targets.time_2m_imagers_planet \
                            + bulge_targets.time_2m_imagers_planet

project_total = sinistro_time_total + floyds_time_total + FT_imagers_time_total
print('Time per instrument class per year:')
print('1m/Sinistro: '+str(sinistro_time_per_year)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_per_year)+'hrs')
print('2m/imagers: '+str(FT_imagers_time_per_year)+'hrs\n')

print('Time per instrument over project lifetime:')
print('1m/Sinistro: '+str(sinistro_time_total)+'hrs')
print('2m/FLOYDS: '+str(floyds_time_total)+'hrs')
print('2m/imagers: '+str(FT_imagers_time_total)+'hrs (Rapid Response mode)\n')

print('Overall project request = '+str(project_total)+'hrs')